# LOAD colocalisations in datasets

In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import pyproj

# from rasterio.transform import Affine

import matplotlib.pyplot as plt
import matplotlib.colors as cl

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.geodesic as cgeo

crs = ccrs.PlateCarree()
import cmocean.cm as cm

# from xgcm import Grid
from xhistogram.xarray import histogram
import warnings

warnings.filterwarnings("ignore")

import os
from glob import glob

/home1/datahome/mdemol/.miniconda3/envs/histenv2/lib/python3.9/site-packages/dask/config.py:742: FutureWarning: Dask configuration key 'distributed.scheduler.transition-log-length' has been deprecated; please use 'distributed.admin.low-level-log-length' instead
  warnings.warn(
/home1/datahome/mdemol/.miniconda3/envs/histenv2/lib/python3.9/site-packages/dask/config.py:742: FutureWarning: Dask configuration key 'distributed.comm.recent-messages-log-length' has been deprecated; please use 'distributed.admin.low-level-log-length' instead
  warnings.warn(


In [2]:
if True:
    from dask.distributed import Client
    from dask_jobqueue import PBSCluster

    cluster = PBSCluster(cores=7, processes=7, walltime="04:00:00")
    w = cluster.scale(jobs=1)
else:
    from dask.distributed import Client, LocalCluster

    #
    cluster = LocalCluster()

client = Client(cluster)
client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: http://10.148.1.54:8787/status,
Dashboard: http://10.148.1.54:8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.148.1.54:53835,Workers: 0
Dashboard: http://10.148.1.54:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


# NEW

In [3]:
colloc_path = {
    2008: "/home/datawork-cersat-public/cache/users/jfpiolle/felyx/drifters/mdb_v2/2008/",
    2009: "/home/datawork-cersat-public/cache/users/jfpiolle/felyx/drifters/mdb_v2/2009/",
    2010: "/home/datawork-cersat-public/cache/users/jfpiolle/felyx/drifters/mdb_v2/2010/",
    2011: "/home/datawork-cersat-public/cache/users/jfpiolle/felyx/drifters/mdb_v2/2011/",
    2012: "/home/datawork-cersat-public/cache/users/jfpiolle/felyx/drifters/mdb_v2/2012/",
    2013: "/home/datawork-cersat-public/cache/users/jfpiolle/felyx/drifters/mdb_v2/2013/",
    2014: "/home/datawork-cersat-public/cache/users/jfpiolle/felyx/drifters/mdb_v2/2014/",
    2015: "/home/datawork-cersat-public/cache/users/jfpiolle/felyx/drifters/mdb_v2/2015/",
    2016: "/home/datawork-cersat-public/cache/users/jfpiolle/felyx/drifters/mdb_v2/2016/",
    2017: "/home/datawork-cersat-public/cache/users/jfpiolle/felyx/drifters/mdb_v2/2017/",
    2018: "/home/datawork-cersat-public/cache/users/jfpiolle/felyx/drifters/mdb_v2/2018/",
    2019: "/home/datawork-cersat-public/cache/users/jfpiolle/felyx/drifters/mdb_v2/2019/",
    2020: "/home/datawork-cersat-public/cache/users/jfpiolle/felyx/drifters/mdb_v2/2020/",
}

In [6]:
def load_collocalisations(source, satellite=None, product_type=None, drifter=None):
    """Load collocalisations altimetry / GDP data

    Parameters
    ----------
    source : str, int
        Data source, e.g. 2016, 2018
    satellite : str
        type of satellite we want to select among "SARAL" or "Sentinel"  (if we want to)
    drifter : str
        type of drifter we want to select among "gps" or "argos" (if we want to)

    Return
    ------
    list of .nc filess

    """
    files = sorted(glob(os.path.join(colloc_path[source], "*.nc")))
    if product_type is not None:
        files = [f for f in files if product_type in f]

    if satellite is not None:
        files = [f for f in files if satellite in f]

    if drifter is not None:
        files = [f for f in files if drifter in f]

    return files

In [7]:
from histlib.cstes import labels
nc_files_new = {
    l: load_collocalisations(
        int(l.split("_")[-1]),
        drifter=l.split("_")[0],
        product_type=l.split("_")[1],
        satellite=l.split("_")[2],
    )
    for l in labels
}
nc_files_new

{'gps_Jason-3_2020': ['/home/datawork-cersat-public/cache/users/jfpiolle/felyx/drifters/mdb_v2/2020/20200101_CMEMS-L3-SLA-Jason-3__gps.nc',
  '/home/datawork-cersat-public/cache/users/jfpiolle/felyx/drifters/mdb_v2/2020/20200201_CMEMS-L3-SLA-Jason-3__gps.nc',
  '/home/datawork-cersat-public/cache/users/jfpiolle/felyx/drifters/mdb_v2/2020/20200301_CMEMS-L3-SLA-Jason-3__gps.nc',
  '/home/datawork-cersat-public/cache/users/jfpiolle/felyx/drifters/mdb_v2/2020/20200401_CMEMS-L3-SLA-Jason-3__gps.nc'],
 'argos_Jason-3_2020': ['/home/datawork-cersat-public/cache/users/jfpiolle/felyx/drifters/mdb_v2/2020/20200101_CMEMS-L3-SLA-Jason-3__argos.nc',
  '/home/datawork-cersat-public/cache/users/jfpiolle/felyx/drifters/mdb_v2/2020/20200201_CMEMS-L3-SLA-Jason-3__argos.nc',
  '/home/datawork-cersat-public/cache/users/jfpiolle/felyx/drifters/mdb_v2/2020/20200301_CMEMS-L3-SLA-Jason-3__argos.nc',
  '/home/datawork-cersat-public/cache/users/jfpiolle/felyx/drifters/mdb_v2/2020/20200401_CMEMS-L3-SLA-Jason-3__

In [8]:
l = 'gps_Jason-2_2010'

In [9]:
cluster.close()